# Differential expression analysis (DEA)

In [ ]:
options(warn=-1)

In [ ]:
library_load <- suppressMessages(
    
    list(
        
        # Seurat 
        library(Seurat), 
        
        # Parallel 
        library(future), 
        
        # GSEA 
        library(msigdbr), 
        library(fgsea), 
        
        # Data 
        library(tidyverse), 
        library(openxlsx), 
        
        # Plotting 
        library(ComplexHeatmap), 
        library(circlize), 
        library(viridis), 
        library(ggplotify), 
        library(patchwork), 
        
        # Python 
        library(reticulate)
    )
)

In [ ]:
# Configure reticulate 
use_condaenv(condaenv="p.3.8.12-FD20200109SPLENO", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
py_config()

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source files
source("plotting_global.R")
source("bin/seurat_qc.R")
source("bin/pbDEA.R")

In [ ]:
# Plotting Theme
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Parameter settings

In [ ]:
# Files 
so_rds_file <- "data/object/pp.rds"

# Import Seurat object

In [ ]:
so <- readRDS(so_rds_file)

# UMAP

In [ ]:
options(repr.plot.width=8, repr.plot.height=8)

dplot_1 <- dplot(so, reduction="umap", group_by="cell_type_fine", alpha=1, pt_size=1.5, label=TRUE, legend_position="none", label_color="black") + scale_color_manual(values=color$cell_type_fine) + ggtitle("Cell type leiden")
dplot_1

# DEA

In [ ]:
grouping_var="cell_type_fine"
pseudobatch_var="sample_group"

p_adj_thr=0.05
sample_weights=FALSE

# Cluster marker  

In [ ]:
# voom lmFit for marker 
results <- voomlmfit_marker(so, grouping_var=grouping_var, pseudobatch_var=pseudobatch_var, sample_weights=FALSE) 

In [ ]:
# Order results 
results <- results[names(color$cell_type_fine)]

## Heatmap 

In [ ]:
options(repr.plot.width=50, repr.plot.height=50)
hm_marker(results, so, grouping_var=grouping_var, width=0.05, height=1)

## Volcano plot

In [ ]:
options(repr.plot.width=5*8, repr.plot.height=1*8)

vp <- lapply(names(results), function(i) {dea_vp(results[[i]], title=i, log2fc_thr=0, p_adj_thr=p_adj_thr)})
ggpubr::ggarrange(plotlist=vp, ncol=5, common.legend=FALSE, legend="bottom")

## Write xlsx

In [ ]:
write_results(results, "result/annotation/marker_dea")